# 🏥 Healthcare Data Exploration with PySpark

This notebook demonstrates loading and exploring Synthea synthetic patient data from Google BigQuery using PySpark.

**Dataset:** CMS Synthetic Patient Data (OMOP format)  
**Source:** `bigquery-public-data.cms_synthetic_patient_data_omop`  
**Size:** 1M+ synthetic patients, ~10GB

---

## 1️⃣ Setup Spark Session

In [ ]:
import sys
sys.path.append('..')  # Add src to path

from src.utils.spark_session import get_spark_session
from src.data_ingestion.bigquery_loader import BigQueryLoader

# Create Spark session
spark = get_spark_session(
    app_name="HealthcareDataExploration",
    local=True
)

print(f"✅ Spark {spark.version} initialized")
print(f"📊 Spark UI: {spark.sparkContext.uiWebUrl}")

## 2️⃣ Load Patient Data

In [ ]:
# Initialize BigQuery loader
loader = BigQueryLoader(
    spark=spark,
    gcp_project="your-project-id"  # CHANGE THIS
)

# Load sample patients (limit for exploration)
patients_df = loader.load_patients(limit=10000)

print(f"\n📊 Loaded {patients_df.count():,} patients")
print(f"📋 Columns: {len(patients_df.columns)}")

In [ ]:
# Display schema
patients_df.printSchema()

In [ ]:
# Show sample records
patients_df.select(
    "person_id",
    "gender_concept_id",
    "year_of_birth",
    "race_concept_id",
    "ethnicity_concept_id",
    "age"
).show(10)

## 3️⃣ Basic Statistics

In [ ]:
from pyspark.sql import functions as F

# Age distribution
print("📊 Age Distribution:")
patients_df.select(
    F.min("age").alias("min_age"),
    F.max("age").alias("max_age"),
    F.avg("age").alias("avg_age"),
    F.stddev("age").alias("std_age")
).show()

# Age groups
print("\n📊 Age Groups:")
patients_df \
    .withColumn("age_group", 
        F.when(F.col("age") < 18, "0-17")
         .when(F.col("age") < 35, "18-34")
         .when(F.col("age") < 50, "35-49")
         .when(F.col("age") < 65, "50-64")
         .otherwise("65+")
    ) \
    .groupBy("age_group") \
    .count() \
    .orderBy("age_group") \
    .show()

## 4️⃣ Load Hospital Encounters

In [ ]:
# Load encounters (hospital visits)
encounters_df = loader.load_encounters(
    limit=50000,
    start_date="2020-01-01"
)

print(f"\n📊 Loaded {encounters_df.count():,} encounters")

In [ ]:
# Show encounters
encounters_df.select(
    "visit_occurrence_id",
    "person_id",
    "visit_start_date",
    "visit_end_date",
    "length_of_stay_days"
).show(10)

In [ ]:
# Length of stay statistics
print("📊 Length of Stay Statistics:")
encounters_df.select(
    F.min("length_of_stay_days").alias("min_los"),
    F.max("length_of_stay_days").alias("max_los"),
    F.avg("length_of_stay_days").alias("avg_los"),
    F.percentile_approx("length_of_stay_days", 0.5).alias("median_los")
).show()

## 5️⃣ Join Patients and Encounters

In [ ]:
# Join patients with their encounters
patient_encounters = patients_df.join(
    encounters_df,
    on="person_id",
    how="inner"
)

print(f"📊 Joined DataFrame: {patient_encounters.count():,} rows")

# Show combined data
patient_encounters.select(
    "person_id",
    "age",
    "gender_concept_id",
    "visit_start_date",
    "length_of_stay_days"
).show(10)

## 6️⃣ Aggregations by Patient

In [ ]:
# Calculate per-patient statistics
patient_stats = patient_encounters.groupBy("person_id", "age") \
    .agg(
        F.count("visit_occurrence_id").alias("total_visits"),
        F.sum("length_of_stay_days").alias("total_los"),
        F.avg("length_of_stay_days").alias("avg_los"),
        F.min("visit_start_date").alias("first_visit"),
        F.max("visit_start_date").alias("last_visit")
    )

patient_stats.orderBy(F.desc("total_visits")).show(10)

print(f"\n📊 Patients with encounters: {patient_stats.count():,}")

## 7️⃣ High Utilizers Analysis

In [ ]:
# Find high utilizers (many hospital visits)
high_utilizers = patient_stats.filter(F.col("total_visits") >= 5)

print(f"🚨 High Utilizers (5+ visits): {high_utilizers.count():,} patients")
print(f"   Percentage: {high_utilizers.count() / patient_stats.count() * 100:.1f}%")

# Show top utilizers
high_utilizers.orderBy(F.desc("total_visits")).show(10)

## 8️⃣ Visualization (Convert to Pandas)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert small aggregated data to Pandas for plotting
visit_counts = patient_stats.groupBy("total_visits") \
    .count() \
    .orderBy("total_visits") \
    .limit(20) \
    .toPandas()

# Plot distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(visit_counts['total_visits'], visit_counts['count'])
plt.xlabel('Number of Visits')
plt.ylabel('Number of Patients')
plt.title('Distribution of Hospital Visits per Patient')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
age_stats = patients_df.groupBy("age") \
    .count() \
    .orderBy("age") \
    .toPandas()
plt.plot(age_stats['age'], age_stats['count'])
plt.xlabel('Age')
plt.ylabel('Number of Patients')
plt.title('Patient Age Distribution')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 9️⃣ Save Processed Data

In [ ]:
# Save to local parquet (for later use)
output_path = "../data/processed/patient_stats.parquet"

patient_stats.write \
    .mode("overwrite") \
    .parquet(output_path)

print(f"✅ Saved patient statistics to: {output_path}")

## 🎯 Summary

In this notebook, we:

1. ✅ Created PySpark session with BigQuery connector
2. ✅ Loaded synthetic patient data (10,000 patients)
3. ✅ Loaded hospital encounters (50,000 visits)
4. ✅ Performed basic exploratory analysis
5. ✅ Joined datasets using PySpark SQL
6. ✅ Calculated per-patient aggregations
7. ✅ Identified high utilizers
8. ✅ Visualized results with Matplotlib
9. ✅ Saved processed data to Parquet

**Next Steps:**
- Explore conditions, procedures, medications
- Feature engineering for ML models
- Build readmission prediction model
- Deploy to GCP Dataproc cluster

---

In [ ]:
# Clean up
spark.stop()
print("🛑 Spark session stopped")